In [1]:
import os
import parmed as pmd
import subprocess as sp

## Process Host files

In [2]:
gaff_version = 'gaff'
host_folder = 'host_files'
host_resname = 'MGO'
host_name = ["bCD"]
host_name += [f"MGLab{i}" for i in [8, 9, 19, 23, 24, 34, 35, 36]]

In [3]:
for host in host_name:
    print(f"Processing host {host}")
    
    # Antechamber
    output = sp.Popen(
        ['antechamber', 
         '-fi', 'mol2', '-i', f'{host}.mol2',
         '-fo', 'mol2', '-o', f'{host}.{gaff_version}.mol2',
         '-at', gaff_version, '-rn', host_resname],
        stdout=sp.PIPE,
        stderr=sp.PIPE,
        cwd=host_folder,
    )
    output = output.stdout.read().decode().splitlines()
    
    # Parmchk2
    output = sp.Popen(
        ['parmchk2',
         '-s', gaff_version,
         '-f', 'mol2',
         '-i', f'{host}.{gaff_version}.mol2',
         '-o', f'{host}.{gaff_version}.frcmod'],
        stdout=sp.PIPE,
        stderr=sp.PIPE,
        cwd=host_folder,
    )
    output = output.stdout.read().decode().splitlines()

    # Vcharges
    output = sp.Popen(
        ['Vcharge_core',
         f'{host}.sdf',
         '-o', f'{host}_vcharge.log',
         '-p', f'{host}_vcharge.sdf',
         '-fc'],
        stdout=sp.PIPE,
        stderr=sp.PIPE,
        cwd=host_folder,
    )
    output = output.stdout.read().decode().splitlines()
    
    # Extract vcharges
    output = sp.Popen(
        ['extractCharges_core',
         f'{host}_vcharge.sdf',
        '-o', f'{host}_vcharge.txt'],
        stdout=sp.PIPE,
        stderr=sp.PIPE,
        cwd=host_folder,
    )
    output = output.stdout.read().decode().splitlines()

    # Update charges in mol2 file
    vcharges = open(os.path.join(host_folder, f'{host}_vcharge.txt'), 'r').readlines()[1:]
    structure = pmd.load_file(os.path.join(host_folder, f'{host}.{gaff_version}.mol2'), structure=True)

    for atom, charge in zip(structure.atoms, vcharges):
        atom.charge = float(charge)

    structure.save(os.path.join(host_folder, f'{host}.vcharge.{gaff_version}.mol2'), overwrite=True)

Processing host bCD
Processing host MGLab8
Processing host MGLab9
Processing host MGLab19
Processing host MGLab23
Processing host MGLab24
Processing host MGLab34
Processing host MGLab35
Processing host MGLab36


## Process Guest files

In [4]:
gaff_version = 'gaff'
guest_folder = 'guest_files'
guest_resname = ['MCH', 'RMT']
guest_name = ['g1', 'g2']

In [5]:
for guest, resname in zip(guest_name, guest_resname):
    print(f"Processing guest {guest}-{resname}")
    
    # Antechamber
    output = sp.Popen(
        ['antechamber', 
         '-fi', 'mol2', '-i', f'{guest}.mol2',
         '-fo', 'mol2', '-o', f'{resname}.{gaff_version}.mol2',
         '-at', gaff_version, '-rn', resname],
        stdout=sp.PIPE,
        stderr=sp.PIPE,
        cwd=guest_folder,
    )
    output = output.stdout.read().decode().splitlines()
    
    # Parmchk2
    output = sp.Popen(
        ['parmchk2',
         '-s', gaff_version,
         '-f', 'mol2',
         '-i', f'{resname}.{gaff_version}.mol2',
         '-o', f'{resname}.{gaff_version}.frcmod'],
        stdout=sp.PIPE,
        stderr=sp.PIPE,
        cwd=guest_folder,
    )
    output = output.stdout.read().decode().splitlines()

    # Vcharges
    output = sp.Popen(
        ['Vcharge_core',
         f'{guest}.sdf',
         '-o', f'{resname}_vcharge.log',
         '-p', f'{resname}_vcharge.sdf',
         '-fc'],
        stdout=sp.PIPE,
        stderr=sp.PIPE,
        cwd=guest_folder,
    )
    output = output.stdout.read().decode().splitlines()
    
    # Extract vcharges
    output = sp.Popen(
        ['extractCharges_core',
         f'{resname}_vcharge.sdf',
        '-o', f'{resname}_vcharge.txt'],
        stdout=sp.PIPE,
        stderr=sp.PIPE,
        cwd=guest_folder,
    )
    output = output.stdout.read().decode().splitlines()

    # Update charges in mol2 file
    vcharges = open(os.path.join(guest_folder, f'{resname}_vcharge.txt'), 'r').readlines()[1:]
    structure = pmd.load_file(os.path.join(guest_folder, f'{resname}.{gaff_version}.mol2'), structure=True)

    for atom, charge in zip(structure.atoms, vcharges):
        atom.charge = float(charge)

    structure.save(os.path.join(guest_folder, f'{resname}.vcharge.{gaff_version}.mol2'), overwrite=True)

Processing guest g1-MCH
Processing guest g2-RMT
